In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

In [2]:
client = QdrantClient(url="https://qdrant-iuhkart.aiclubiuh.com/")

In [6]:
embedding_size = 384
distance = Distance.COSINE
client.create_collection(
    collection_name='Production',
    vectors_config=VectorParams(size=embedding_size, distance=distance)
)

ResponseHandlingException: timed out

In [4]:
qd.reinit_collection(collection_name="product")

NameError: name 'qd' is not defined

In [1]:
import sqlite3
import pandas as pd
path_sqlite3 = "/Users/qxnam/Documents/semeter/DATA-PLATFORM/IUHKart/iuhkart/db.sqlite3"
conn = sqlite3.connect(path_sqlite3)

query_promt = '''
SELECT d.slug, d.product_name, c.category_name
FROM product d 
JOIN category c ON c.category_id = d.category_id
'''

df = pd.read_sql_query(query_promt, conn)

In [5]:
qd.insert(df)

Inserting: 100%|██████████| 146/146 [00:17<00:00,  8.35it/s, Status=✅ Success]


In [9]:
qd.search(prompt='women', limit=2)

[{'id': 'a82d6f71-75b8-4efd-a67d-d76e26983bcd',
  'version': 0,
  'score': 0.49265187379202524,
  'payload': {'slug': 'womens-shoulder-canvas-bag-large-with-chain-capacity-for-women-on-offer-2023-new',
   'product_name': "Women's Shoulder Canvas Bag Large With Chain Capacity for\xa0women on offer 2023\xa0new",
   'category_name': 'Luggages & Bags'},
  'vector': None,
  'shard_key': None},
 {'id': '749fdfe4-637a-40da-af6d-2330cd79a368',
  'version': 0,
  'score': 0.4602108656855013,
  'payload': {'slug': 'women-canvas-messenger-bag-youth-ladies-fashion-shoulder-bag-student-large-capacity-female-crossbody-bags-woman-packet',
   'product_name': 'Women Canvas Messenger Bag Youth Ladies Fashion Shoulder Bag Student Large Capacity Female Crossbody Bags Woman Packet',
   'category_name': 'Luggages & Bags'},
  'vector': None,
  'shard_key': None}]

In [4]:
qd.close()

👋 bye
